# Example Usage of Scraipe Library

Here's a quick example using scraipe to extract mentions of celebrities in news articles.

## Setup
Install and import dependencies

In [1]:
%pip install scraipe
import sys; sys.path.append('scraipe')

ERROR: Could not find a version that satisfies the requirement scraipe (from versions: none)
ERROR: No matching distribution found for scraipe
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from scraipe import Workflow
from scraipe.scrapers import NewsScraper
from scraipe.analyzers import OpenAiAnalyzer
from pydantic import BaseModel

## Extract links
First, we need a list of links to target with scraipe. We will extract all links from the front page of https://apnews.com.

In [3]:
import requests
import re

url = "https://apnews.com/"
response = requests.get(url)
html = response.text

# Use a regex to extract article links
pattern = r'href=["\'](?=[^"\']*/article)([^"\']+)["\']'
news_links = re.findall(pattern, html)

# Remove duplicates
news_links = list(set(news_links))

# Display a summary of the links
news_links_df = pd.DataFrame(news_links, columns=['link'])
print(f"Found {len(news_links_df)} news links")
display(news_links_df.head())

Found 140 news links


,link
0,https://apnews.com/article/mia-love-brain-canc...
1,https://apnews.com/article/texas-university-am...
2,https://apnews.com/article/work-life-balance-s...
3,https://apnews.com/article/mexico-desaparecido...
4,https://apnews.com/article/trump-america-boyco...


## Configure Workflow
Now we'll configure the scraipe workflow using DefaultScraper and OpenAiAnalyzer.

Store your OpenAI key in a file named .openai_key before running this code block.

In [ ]:
#===Configure NewsScraper===
# NewsScraper doesn't require any extra configuration
scraper = NewsScraper()

#===Configure OpenAiAnalyzer===
# Load API key from a file
api_key = open(".openai_key").read().strip()

# Define the instruction for the LLM. Ensure the instruction specifies a return schema.
instruction = '''Extract a list of celebrities mentioned in the article text.
Return a JSON dictionary with the following schema:
{"celebrities":["celebrity1", "celebrity2", ...]}'''

# (Optional) Create a pydantic schema to validate the LLM output
from typing import List
class ExpectedOutput(BaseModel):
    celebrities: List[str]
    
# Create the analyzer with the API key, instruction, and schema
analyzer = OpenAiAnalyzer(api_key,instruction,pydantic_schema=ExpectedOutput)

#===Create Workflow===
# Create a workflow with the configured scraper and analyzer
workflow = Workflow(scraper, analyzer)

## Scrape content from news links
Next we will scrape content from news links. This content will be saved within the workflow's scrape store.

In [5]:
# Scrape the news links
workflow.scrape(news_links)
# Display the scraped content
scrape_store_df = workflow.get_scrapes()
display(scrape_store_df.iloc[0:1])

Scraping 140/140 new or failed links...


Scraping URLs: 100%|██████████| 140/140 [01:03<00:00,  2.20it/s]

Successfully scraped 140/140 links.


,link,content,success,error
0,https://apnews.com/article/mia-love-brain-canc...,SALT LAKE CITY (AP) — The death Sunday of form...,True,None


## Analyze content with OpenAI
Next we will analyze the stored scrapes.

In [6]:
# Analyze the scraped content
workflow.analyze()

# Dispaly the analyses
analysis_store_df = workflow.get_analyses()
display(analysis_store_df.head())

Analyzing 140/140 new or failed links...


Analyzing content: 100%|██████████| 140/140 [02:24<00:00,  1.03s/it]

Successfully analyzed 140/140 links.


,link,output,success,error
0,https://apnews.com/article/mia-love-brain-canc...,"{'celebrities': ['Mia Love', 'Barack Obama', '...",True,None
1,https://apnews.com/article/texas-university-am...,"{'celebrities': ['George H.W. Bush', 'Donald T...",True,None
2,https://apnews.com/article/work-life-balance-s...,"{'celebrities': ['Justin Stewart', 'Bobby Dutt...",True,None
3,https://apnews.com/article/mexico-desaparecido...,"{'celebrities': ['Omar García Harfuch', 'Donal...",True,None
4,https://apnews.com/article/trump-america-boyco...,"{'celebrities': ['Donald Trump', 'Jeff Bezos',...",True,None


## Compile the results
Finally, let's compile a pretty dataset for export. 

In [18]:
# Get the complete store of records
records_df = workflow.get_records()

# Create a dataframe with only the link and a comma-delimited nested list of extracted celebrities
export_df = records_df[['link']].copy()

export_df['celebrities'] = records_df['output'].apply(lambda x: x['celebrities'])

display(export_df.head())

# Export the extacted celebrity data
export_df.to_csv('celebrities.csv', index=False)


,link,celebrities
0,https://apnews.com/article/mia-love-brain-canc...,"[Mia Love, Barack Obama, Donald Trump, Joe Bid..."
1,https://apnews.com/article/texas-university-am...,"[George H.W. Bush, Donald Trump]"
2,https://apnews.com/article/work-life-balance-s...,"[Justin Stewart, Bobby Dutton, Amber Krasinski..."
3,https://apnews.com/article/mexico-desaparecido...,"[Omar García Harfuch, Donald Trump, David Sauc..."
4,https://apnews.com/article/trump-america-boyco...,"[Donald Trump, Jeff Bezos, Kamala Harris, Geor..."


## Analyze the results
Now you can conduct your own analysis on the struc

In [22]:
# Load the extracted data
celebrities_df = pd.read_csv('celebrities.csv')
from ast import literal_eval
celebrities_df['celebrities'] = celebrities_df['celebrities'].apply(literal_eval)

# Explode the nested list of celebrities
celebrities_df = celebrities_df.explode('celebrities')
celebrities_df['celebrities'] = celebrities_df['celebrities'].str.strip()

# Display the top 10 most mentioned celebrities
celebrities_df = celebrities_df['celebrities'].value_counts().reset_index()
celebrities_df.columns = ['celebrity', 'mentions']
celebrities_df = celebrities_df.sort_values('mentions', ascending=False)
celebrities_df.head(10)

,celebrity,mentions
0,Donald Trump,52
1,Elon Musk,19
2,Joe Biden,6
7,Barack Obama,4
6,Robert F. Kennedy Jr.,4
5,Kamala Harris,4
4,Vladimir Putin,4
3,Marco Rubio,4
11,George W. Bush,3
10,John Roberts,3
